In [1]:
import numpy as np
import pandas as pd
import requests
from pykrx import stock
import time
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

name = '카카오'
fromdate = str(20180101)
todate = str(20211104)

ticker1 = stock_list.loc[stock_list['종목명']== name, '종목코드']
df1 = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker1)

seri = df1['종가']
df = pd.DataFrame(seri, columns=['종가','일계도','이계도','일계도부호','이계도부호'])

df['일계도'][0] = 0
for i in range(1, len(seri)):
    df['일계도'][i] = df['종가'][i] - df['종가'][i-1]

df['이계도'][0] = 0
for i in range(1, len(seri)):
    df['이계도'][i] = df['일계도'][i] - df['일계도'][i-1]

for i in range(len(df)):
    if df['일계도'][i] >= 0:
        df['일계도부호'][i] = 1
    else:
        df['일계도부호'][i] = 0

for i in range(len(df)):
    if df['이계도'][i] >= 0:
        df['이계도부호'][i] = 1
    else:
        df['이계도부호'][i] = 0

df_diff2 = df['일계도부호']
df_diff2 = np.array(df_diff2)

C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:39:

In [6]:
train_X = np.array([df_diff2[i:i+30] for i in range(len(df_diff2[:700]))])
train_Y = np.array([df_diff2[i+31] for i in range(len(df_diff2[:700]))])

In [7]:
test_X = np.array([df_diff2[i:i+30] for i in range(len(df_diff2[700:880]))])
test_Y = np.array([df_diff2[i+31] for i in range(len(df_diff2[700:880]))])

In [8]:
train_Y = train_Y.reshape(700,1)
test_Y = test_Y.reshape(180,1)

In [19]:
train_X.shape, train_Y.shape

((700, 30), (700, 1))

In [10]:
train_X = train_X.reshape(train_X.shape[0], 30, 1)
test_X = test_X.reshape(test_X.shape[0], 30, 1)

In [11]:
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping

K.clear_session()
    
model = Sequential() # Sequeatial Model 
model.add(LSTM(20, input_shape=(30, 1))) # (timestep, feature) 
model.add(Dense(1)) # output = 1 
model.compile(loss='mean_squared_error', optimizer='adam') 
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20)                1760      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [12]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(train_X, train_Y, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ghdak\AppData\Local\Temp/ipykernel_8604/3955987928.py", line 4, in <module>
    batch_size=30, verbose=1, callbacks=[early_stop])
  File "C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 728, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 224, in fit
    distribution_strategy=strategy)
  File "C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 547, in _process_training_inputs
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\ghdak\anaconda3\envs\dl\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line

In [32]:
import import_ipynb
from momentum import model

In [33]:
pred_y = model.predict(test_x)
pred_y

C:\Users\ghdak\anaconda3\envs\gpu_env\lib\site-packages\sklearn\base.py:561: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


array([ 0.26768783,  0.41374824,  0.22665378,  0.43345531,  0.31884867,
        0.38649868,  0.62158501,  0.8373333 ,  0.47945128,  0.95364879,
        0.41451476,  0.43648125,  0.6847098 ,  0.48067746,  0.88978416,
        0.42086582,  0.50391141,  0.6474081 ,  0.24804938,  0.32638992,
        0.585119  ,  0.7596226 ,  0.62993814,  0.38199133,  0.48475597,
        0.48088229,  0.28453316,  0.33873695,  0.46849417,  0.64547477,
        0.74703992,  0.48587066,  0.63989393,  0.43683798,  0.61284096,
        0.67160984,  0.85008405,  0.94525221,  0.47485962,  0.64192948,
        0.36555496,  0.66839314,  0.27696661,  0.49880123,  0.71271825,
        0.7559414 ,  0.57747873,  0.37218869,  0.30207655,  0.51623011,
        0.53503195,  0.35026355,  0.41952344,  0.33251302,  0.28733416,
        0.53990351,  0.41698558,  0.76757478,  0.59796604,  0.73865361,
        0.96011338,  0.83639161,  0.63678196,  0.66694729,  0.3670489 ,
        0.30605897,  0.3164337 ,  0.31982371,  0.14811374,  0.37

In [36]:
trans_pred = pred_y.copy()
for i in range(len(pred_y)):
    if pred_y[i] < 0.5:
        trans_pred[i] = 0
    else:
        trans_pred[i] = 1

In [37]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_y, trans_pred)
round(mae, 2)

0.42